In [1]:
import numpy as np
import json
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
train_data = []

with open('domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data.append(data)

In [3]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

/var/folders/mz/3ys1k7ln271968b9ty631lfw0000gn/T/ipykernel_56448/1473323008.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array([entry["text"] for entry in train_data])


In [4]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# BOW n-grams
#vectorizer = HashingVectorizer(n_features=10000)
vectorizer = CountVectorizer(ngram_range=(1,2))

X_train_text = [' '.join(map(str, sample)) for sample in X_train]
X_test_text = [' '.join(map(str, sample)) for sample in X_test]

X_train_transformed = vectorizer.fit_transform(X_train_text)
X_test_transformed = vectorizer.transform(X_test_text)

In [27]:
logistic_model = LogisticRegression(
    penalty="l2",
    # solver="saga",
    max_iter=5000
)

### 插入部分

In [24]:
param_grid = {'C': [0.1, 1, 10]}
# grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

In [25]:
grid_search = GridSearchCV(logistic_model, param_grid, cv=10)

In [26]:
grid_search.fit(X_train_transformed, y_train)

/Users/haoqian/anaconda3/envs/CV/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/haoqian/anaconda3/envs/CV/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/haoqian/anaconda3/envs/CV/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/haoqian/anaconda3/envs/CV/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/haoqian/anaconda3/envs/CV/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


KeyboardInterrupt: 

In [19]:
best_model = grid_search.best_estimator_
# best_model.fit(X_train_transformed, y_train)


In [20]:
y_pred = best_model.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")
# kaggle结果76.8%

Test Accuracy: 0.9302564102564103


### 插入结束

In [20]:
# logistic_model.fit(X_train_transformed, y_train)

LogisticRegression()

In [33]:
y_pred = best_model.predict(X_test_transformed)

# output acc
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9302564102564103


In [21]:
with open('logistic_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  

    # read in dataset
    with open('test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # word embedding
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # use logistic model to predict
            prediction = best_model.predict(X_test)

            # output csv
            output_file.write(f"{entry['id']},{prediction[0]}\n")